<a href="https://colab.research.google.com/github/JOSHITHA6/SPPECH_2_TEXT/blob/main/SPEECH_TO_TEXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install ffmpeg
!pip install transformers torchaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
import requests

# Raw URL of the file in your GitHub repository
url = 'https://raw.githubusercontent.com/JOSHITHA6/SPPECH_2_TEXT/8cdf09c264a95c478e1cde09f894c0088d2ad268/Bicky_better.m4a'

# Path to save the downloaded file
audio_path = '/content/Bicky_better.m4a'

# Download the file from GitHub
response = requests.get(url)

# Write the content to a file
with open(audio_path, 'wb') as file:
    file.write(response.content)

print(f"File downloaded and saved to {audio_path}")


File downloaded and saved to /content/Bicky_better.m4a


In [3]:
import os

# Path to your M4A file
audio_path = "/content/Bicky_better.m4a"

# Verify the uploaded file exists
print(f"Audio file path: {audio_path}")
print("File exists:", os.path.exists(audio_path))



Audio file path: /content/Bicky_better.m4a
File exists: True


In [4]:
# Set the output file name for WAV
output_file = "/content/converted_audio.wav"

# Convert M4A to WAV using FFmpeg
os.system(f"ffmpeg -i {audio_path} -ar 16000 -ac 1 {output_file}")
print(f"Audio converted and saved as: {output_file}")

Audio converted and saved as: /content/converted_audio.wav


In [6]:
from IPython.display import Audio

# URL of the audio file on GitHub
audio_url = 'https://raw.githubusercontent.com/JOSHITHA6/SPPECH_2_TEXT/8cdf09c264a95c478e1cde09f894c0088d2ad268/Bicky_better.m4a'

# Play the audio from the URL
Audio(audio_url)


In [9]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio
import os

# Set the file paths
audio_path = "/content/Bicky_better.m4a"
output_file = "/content/converted_audio.wav"

# Convert M4A to WAV using FFmpeg
os.system(f"ffmpeg -i {audio_path} -ar 16000 -ac 1 {output_file}")
print(f"Audio converted and saved as: {output_file}")

# Load the Whisper model and processor
model_name = "openai/whisper-large"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Load and preprocess the audio
speech_array, sampling_rate = torchaudio.load(output_file)
print("Audio loaded. Sampling rate:", sampling_rate)

# Resample if necessary (Whisper expects 16kHz)
if sampling_rate != 16000:
    print("Resampling audio to 16kHz...")
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    speech_array = resampler(speech_array).squeeze(0)
    print("Resampled Shape:", speech_array.shape)

# Make sure the audio is in the right shape for Whisper (single channel, 16kHz)
speech_array = speech_array.squeeze()  # Ensure it's a 1D tensor

# Prepare the audio for the model
input_features = processor(speech_array, sampling_rate=16000, return_tensors="pt").input_features

# Perform transcription
with torch.no_grad():
    predicted_ids = model.generate(input_features)



Audio converted and saved as: /content/converted_audio.wav
Audio loaded. Sampling rate: 16000


In [8]:

# Decode the predicted text
transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)


print("TRANSCRIBED TEXT:", transcription)

TRANSCRIBED TEXT:  Bikki went to buy some butter, but the butter was so better. So she went to buy the bitter butter, but the bitter butter was so better.


In [10]:
#SENTIMENT ANALYSIS
# Use libraries like TextBlob or Transformers to perform sentiment analysis on the transcribed text.
from textblob import TextBlob

# Perform sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity  # Range: -1 (negative) to 1 (positive)
    return sentiment

transcribed_text = "transcription"  # Replace with your transcription
sentiment = analyze_sentiment(transcribed_text)

if sentiment > 0:
    print("The sentiment is Positive.")
elif sentiment < 0:
    print("The sentiment is Negative.")
else:
    print("The sentiment is Neutral.")


The sentiment is Neutral.


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import nltk
nltk.download('punkt_tab') # Download the punkt_tab data package

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
!pip install rake-nltk # Install the rake-nltk package using pip

In [15]:
# Import necessary libraries for RAKE
from rake_nltk import Rake

# Initialize RAKE and extract keywords
rake = Rake()

# Extract keywords from the transcribed text
rake.extract_keywords_from_text(transcription)

# Get the ranked phrases (keywords)
keywords = rake.get_ranked_phrases()

# Print the extracted keywords
print("Extracted Keywords: ", keywords)

Extracted Keywords:  ['bitter butter', 'bitter butter', 'bikki went', 'went', 'butter', 'butter', 'buy', 'buy', 'better', 'better']


In [16]:
# Remove duplicates by converting the list to a set
unique_keywords = list(set(keywords))

# Print the unique extracted keywords
print("Unique Extracted Keywords:", unique_keywords)


Unique Extracted Keywords: ['butter', 'bikki went', 'buy', 'bitter butter', 'went', 'better']
